In [8]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
cat_df = pd.read_csv('labelled_data.csv')
cat_df.head()

,img_name,features,left_eye_x,left_eye_y,right_eye_x,right_eye_y,mouth_x,mouth_y,left_ear_1_x,left_ear_1_y,left_ear_2_x,left_ear_2_y,left_ear_3_x,left_ear_3_y,right_ear_1_x,right_ear_1_y,right_ear_2_x,right_ear_2_y,right_ear_3_x,right_ear_3_y
0,00000001_000.jpg,9,175,160,239,162,199,199,149,121,137,78,166,93,281,101,312,96,296,133
1,00000001_005.jpg,9,96,96,153,127,103,146,48,20,78,-37,127,5,210,52,260,63,229,121
2,00000001_008.jpg,9,318,222,340,224,327,238,301,199,304,182,321,196,341,197,359,183,361,203
3,00000001_011.jpg,9,167,173,242,185,191,247,112,105,118,0,171,74,270,90,353,35,332,147
4,00000001_012.jpg,9,115,122,179,121,133,169,95,70,93,17,131,46,233,39,281,9,224,69


In [13]:
n = 9
img_name = cat_df.iloc[n, 0]
points = cat_df.iloc[n, 2:]
points = np.asarray(points)
points = points.astype('float').reshape(-1, 2)

print('Image name: {}'.format(img_name))
print('Landmarks shape: {}'.format(points.shape))
print('First 4 Landmarks: {}'.format(points[:4]))

Image name: 00000001_027.jpg
Landmarks shape: (9, 2)
First 4 Landmarks: [[229. 142.]
 [273. 147.]
 [247. 171.]
 [210. 109.]]


In [14]:
def show_landmarks(image, landmarks):
    """Show image with landmarks"""
    plt.imshow(image)
    plt.scatter(points[:, 0], points[:, 1], s=10, marker='.', c='r')
    plt.pause(0.001)  # pause a bit so that plots are updated

plt.figure()
show_landmarks(io.imread(os.path.join('cats/', img_name)),
               points)
plt.show()

NameError: name 'plt' is not defined